In [19]:
import torch
import cv2
from data.MyDataSet import MyDataSet
from torch.utils.data import DataLoader
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

In [40]:
def predictImage(imgPath, device, U_Net_Model, YOLO_Model):
    imgSize = (512, 512)
    inputImg = cv2.imread(imgPath)
    inputImg = cv2.resize(inputImg, imgSize)
    tmp_inputImg = inputImg
    tmp_inputImg = cv2.cvtColor(tmp_inputImg, cv2.COLOR_BGR2GRAY)
    tmp_inputImg = tmp_inputImg.reshape(1, tmp_inputImg.shape[0], tmp_inputImg.shape[1])
    tmp_inputImg = torch.from_numpy(tmp_inputImg)
    tmp_inputImg = tmp_inputImg.unsqueeze(0)
    tmp_inputImg = tmp_inputImg.to(device=device, dtype=torch.float32)
    with torch.no_grad():
        pred = U_Net_Model(tmp_inputImg)
        
        # prepare for storage
        pred = np.array(pred.data.cpu()[0])[0]

        pred[pred >= 0.5] = 255
        pred[pred < 0.5] = 0

        non_zero_indices = np.nonzero(pred)
        non_zero_values = pred[non_zero_indices]

        mask = np.zeros_like(inputImg)

        # Set the labeled points to a color (e.g., red)
        color = (0, 255, 0)  # BGR format
        for y, x in zip(non_zero_indices[0], non_zero_indices[1]):
            mask[y, x] = color

    yoloResult = YOLO_Model.predict(imgPath,imgsz=512, conf=0.1)  # predict on an image
    # Show the results
    for r in yoloResult:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im_array = np.array(im_array)
        im_array = cv2.resize(im_array, imgSize)

    # Add the mask to the original image
    return cv2.addWeighted(im_array, 1, mask, 0.5, 0)  # Adjust the alpha value for transparency

In [28]:
YOLO_Path = "./weights/best.pt"
U_Net_Path = "./model.mod"

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
U_Net_Model = torch.load(U_Net_Path)
U_Net_Model.eval()
U_Net_Model = U_Net_Model.to(device)
# Load a model
YOLO_Model = YOLO(YOLO_Path)  # load a pretrained model (recommended for training)

## Predict One Image

In [41]:
imgPath = "./images/test2.jpg"
result = predictImage(imgPath, device, U_Net_Model, YOLO_Model)
# Save the result
output_path = "combined.jpg"
cv2.imwrite(output_path, result)


image 1/1 /mnt/d/RHIT/CSSE463_Image_Recognition/Project/CSSE463_Project_Car_Lane_Detection/Test/images/test2.jpg: 288x512 3 cars, 7.2ms
Speed: 1.0ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 512)


True

## Predict Many Images

In [47]:
ROOT_PATH ="./images/05081544_0305/"
OUTPUT_ROOT_PATH = "./images/output/"
imgListPath = "./images/05081544_0305/imgList.txt"
with open(imgListPath) as f:
    imgPaths = [line.rstrip('\n') for line in f]

for imgPath in imgPaths:
    inputPath = os.path.join(ROOT_PATH, imgPath)
    outputPath = os.path.join(OUTPUT_ROOT_PATH, imgPath)
    result = predictImage(inputPath, device, U_Net_Model, YOLO_Model)
    cv2.imwrite(outputPath, result)


image 1/1 /mnt/d/RHIT/CSSE463_Image_Recognition/Project/CSSE463_Project_Car_Lane_Detection/Test/images/05081544_0305/05081544_0305-000001.jpg: 192x512 11 cars, 1 truck, 8.6ms
Speed: 1.2ms preprocess, 8.6ms inference, 4.3ms postprocess per image at shape (1, 3, 192, 512)

image 1/1 /mnt/d/RHIT/CSSE463_Image_Recognition/Project/CSSE463_Project_Car_Lane_Detection/Test/images/05081544_0305/05081544_0305-000002.jpg: 192x512 13 cars, 5.9ms
Speed: 0.8ms preprocess, 5.9ms inference, 1.4ms postprocess per image at shape (1, 3, 192, 512)

image 1/1 /mnt/d/RHIT/CSSE463_Image_Recognition/Project/CSSE463_Project_Car_Lane_Detection/Test/images/05081544_0305/05081544_0305-000003.jpg: 192x512 15 cars, 1 truck, 6.3ms
Speed: 0.7ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 192, 512)

image 1/1 /mnt/d/RHIT/CSSE463_Image_Recognition/Project/CSSE463_Project_Car_Lane_Detection/Test/images/05081544_0305/05081544_0305-000004.jpg: 192x512 12 cars, 4 trucks, 5.8ms
Speed: 0.8ms pre

In [49]:
# Set the path to the directory containing your images
images_directory = './images/output/'

# Get the list of image filenames in the directory
image_filenames = sorted(os.listdir(images_directory))

# Specify the output video filename
output_video_filename = 'output_video.mp4'

# Define the frame size (width, height) for the video
frame_size = (512, 512)  # Adjust the size as needed

# Create a video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter(output_video_filename, fourcc, 30, frame_size)

# Loop through image filenames and write frames to the video
for image_filename in image_filenames:
    image_path = os.path.join(images_directory, image_filename)
    image = cv2.imread(image_path)
    if image is not None:
        out.write(image)

# Release the video writer
out.release()

print("Video created:", output_video_filename)

Video created: output_video.mp4
